### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

import gym
from gymnasium import spaces

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


from collections import deque
import random

### Download Data

In [3]:
from finrl import config_tickers
df = YahooDownloader(start_date = '2009-01-01',
                           end_date = '2020-09-30',
                           ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (86111, 8)


### Process Data

In [4]:
df = FeatureEngineer(use_technical_indicator=True,
                      tech_indicator_list = config.INDICATORS,
                      use_turbulence=True,
                      user_defined_feature = False).preprocess_data(df.copy()).fillna(0)

Successfully added technical indicators
Successfully added turbulence index


In [5]:
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]
df = df.reset_index(drop=True)

In [6]:
df.sample(5)

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
32865,2013-07-05,57.310318,63.950001,63.110001,63.880001,5645800,DIS,4,-0.162581,58.612405,55.622085,52.569485,-28.045625,10.574675,57.409394,57.367637,19.636359
26541,2012-08-20,12.845260,19.150000,18.799999,19.000000,39384600,CSCO,0,0.401166,13.075960,9.753195,61.582303,199.254991,41.102902,11.301655,11.307683,18.122325
48981,2015-09-18,25.557520,28.575001,27.967501,28.052500,297141200,AAPL,4,-0.063514,26.785279,23.351842,46.790692,12.198735,28.649704,25.395998,26.623536,25.679501
28749,2012-12-10,47.863419,64.339996,63.029999,64.339996,8150000,HD,0,0.564217,50.256710,46.218327,55.177859,31.008171,8.007257,47.653019,46.661244,24.683903
14496,2010-12-27,15.974270,17.792500,17.162500,17.170000,31037600,V,0,-0.475950,18.996602,14.543501,44.424284,-105.750703,17.276370,16.887273,17.159716,16.240125


### Train-Test Split

In [7]:
train = data_split(df, '2009-01-01','2019-12-31')
trade = data_split(df, '2020-01-01','2020-09-30')

### Environment Setup

In [8]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension #stock, shares, technical indicators, 

In [9]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
env_kwargs = {
    "hmax": 100,  # Maximum number of shares that can be bought or sold in a single transaction.
    "initial_amount": 1_000_000,  # Initial cash available for the agent to trade with (e.g., $1,000,000).
    "num_stock_shares": num_stock_shares,  # Initial portfolio: number of shares owned for each stock.
    "buy_cost_pct": buy_cost_list,  # Transaction cost percentage for buying stocks.
    "sell_cost_pct": sell_cost_list,  # Transaction cost percentage for selling stocks.
    "state_space": state_space,  # Dimension of the state space (e.g., features describing the environment).
    "stock_dim": stock_dimension,  # Number of stocks being traded (dimension of the stock universe).
    "tech_indicator_list": config.INDICATORS,  # List of technical indicators used as features for the state space.
    "action_space": stock_dimension,  # Dimension of the action space (one action per stock).
    "reward_scaling": 1e-4,  # Scaling factor for rewards to normalize them and improve learning stability.
    "print_verbosity":5
}

### Custom Trading Environment

In [10]:
class CustomStockTradingEnv(StockTradingEnv):
    def __init__(self, df, **kwargs):
            # Call the parent constructor
            super().__init__(df=df, **kwargs)
            self.action_space = spaces.Box(
            low=-self.hmax, 
            high=self.hmax, 
            shape=(self.stock_dim,), 
            dtype=np.float32
        )
    def save_state_memory(self):
        """Save state memory dynamically without hard-coded column names."""
        if len(self.df.tic.unique()) > 1:
            # Get the date memory (excluding the last one)
            date_list = self.date_memory[:-1]

            # Ensure state_list is not empty
            state_list = self.state_memory
            # print(len(state_list[0]))
            if not state_list:
                print("Warning: state_memory is empty!")
                return pd.DataFrame()  # Return an empty DataFrame if no data exists

            # Ensure each state has the same structure
            try:
                num_features = len(state_list[0])  # Get number of features from the first state
                state_columns = [f"feature_{i}" for i in range(num_features)]  # Generate feature names
            except IndexError:
                print("Error: state_memory contains empty entries.")
                return pd.DataFrame()  # Return an empty DataFrame if there's an issue

            df_date = pd.DataFrame(date_list, columns=["date"])
            df_states = pd.DataFrame(state_list, columns=state_columns)
            df_states.index = df_date["date"]  # Set index to match dates
        else:
            date_list = self.date_memory[:-1]
            state_list = self.state_memory
            df_states = pd.DataFrame({"date": date_list, "states": state_list})

        return df_states
    # def step(self, action):
    #     # Clip actions to feasible shares based on current holdings
    #     # print(action)
    #     for i in range(self.stock_dim):
    #         current_shares = self.state[i * 2 + 1]
    #         action[i] = np.clip(action[i], -current_shares, self.hmax)
        
    #     return super().step(action)
    def step(self, action, state = None):
        # Clip actions to feasible shares based on current holdings
        if state is not None:
            self.state = state
        for i in range(self.stock_dim):
            current_shares = self.state[i * 2 + 1]
            action[i] = np.clip(action[i], -current_shares, self.hmax)
        
        return super().step(action)
    

### Multi-Layer Perceptron

In [11]:
class MetaPolicy(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, hmax=100):
        """
        :param hmax: Maximum shares allowed (used for action scaling)
        """
        super().__init__()
        self.hmax = hmax
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Tanh()  # Outputs in [-1, 1] for easy scaling
        )
    
    def forward(self, x):
        # Outputs in [-1, 1], scaled to [-hmax, hmax]
        return self.net(x) * self.hmax

### Define Meta Environment

In [12]:
meta_env_train_gym = CustomStockTradingEnv(df=trade, **env_kwargs)
meta_env_train, obs = meta_env_train_gym.get_sb_env()
# print(obs)
num_stocks  = meta_env_train_gym.action_space.shape[0]
state_features_dim = meta_env_train_gym.state_space
num_base_models = 3
input_dim = num_base_models * num_stocks + state_features_dim
output_dim = num_stocks
hidden_dim = 64 

### Get Base Models

In [13]:
agent = DRLAgent(env = meta_env_train)
model_ddpg = agent.get_model("ddpg")
trained_ddpg = model_ddpg.load(config.TRAINED_MODEL_DIR + "/agent_ddpg")
model_ppo = agent.get_model("ppo")
trained_ppo = model_ppo.load(config.TRAINED_MODEL_DIR + "/agent_ppo", device='cpu')
model_a2c = agent.get_model("a2c")
trained_a2c = model_a2c.load(config.TRAINED_MODEL_DIR + "/agent_a2c", device='cpu')

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device
{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cuda device
{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device


c:\Users\natna\miniforge3\envs\finRl\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
c:\Users\natna\miniforge3\envs\finRl\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CU

### Define Meta Model

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
meta_policy_model = MetaPolicy(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(meta_policy_model.parameters(), lr=1e-3)

In [16]:
# Initialize replay buffer with a maximum capacity (e.g., 10,000 transitions)
replay_buffer = deque(maxlen=10000)
batch_size = 64  # Number of transitions to sample per training step

### Training (First Iteration)

1. Meta-Policy Input Formation:
At each step, the algorithm first queries each of your three pre-trained base RL models using the current environment state. These models return their respective action vectors. These outputs are then concatenated with the current state features to form a single input vector. This combined input is intended to capture both the “opinions” of the base models and the actual market or environment context.

2. Meta-Policy Decision:
This input vector is fed into your meta-policy neural network (an MLP), which produces a final action vector. Think of the meta-policy as a decision maker that learns to weigh the different base models' recommendations together with the actual state of the market to decide on the best trading action.

3. Environment Interaction & Experience Collection:
The final action produced by the meta-policy is executed in the trading environment. The environment returns the next state, the reward for the action, and whether the episode has ended. This transition—comprising the input vector, the action taken, the reward received, the next state, and a done flag—is stored in a replay buffer. This buffer accumulates experiences over time.

4. Online Learning via Experience Replay:
Once the replay buffer has collected enough samples, the algorithm randomly samples a batch of experiences from it. Using this batch, it performs a training update on the meta-policy network. The loss function in this update is designed (in a policy gradient style) to encourage the network to produce actions that are correlated with higher rewards. In other words, the network is nudged in the direction of decisions that historically resulted in better outcomes.

5. Continuous Improvement:
This process repeats over many episodes. As the meta-policy network is updated with more experience, it gradually learns to better combine the base models’ outputs and state information to produce final actions that maximize rewards in the trading environment.


In [72]:
for episode in range(100):
    state = meta_env_train.reset()
    done = False
    
    while not done:
        # 1. Get base model predictions for CURRENT state
        state = np.array(state).reshape(-1)
        base_actions = []
        for model in [trained_ddpg, trained_ppo, trained_a2c]:
            action, _ = model.predict(state)  # Use current state
            base_actions.append(action)
        base_actions = np.concatenate(base_actions).reshape(-1)
        
        # 2. Prepare meta-policy input
        input_vector = np.concatenate([base_actions, state])
        input_tensor = torch.FloatTensor(input_vector).unsqueeze(0).to(device)
        
        # 3. Predict action (e.g., [-50, 0, 100])
        with torch.no_grad():
            predicted_action = meta_policy_model(input_tensor)
        action = predicted_action.cpu().numpy()
        # print(predicted_action.shape)
        # 4. Take action in the environment
        next_state, reward, done, info = meta_env_train.step(action)
        # print(len(next_state))
        # 5. Store transition in replay buffer
        replay_buffer.append((input_vector, action, reward, next_state, done))
        
        # 6. Batch training (only if buffer has enough samples)
        if len(replay_buffer) >= batch_size:
            # Sample a batch of transitions
            batch = random.sample(replay_buffer, batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)
            # print(len(states[0]))
            # Convert to tensors
            states_tensor = torch.FloatTensor(np.array(states)).to(device)
            actions_tensor = torch.FloatTensor(np.array(actions)).to(device)
            rewards_tensor = torch.FloatTensor(np.array(rewards)).to(device)
            
            # 7. Calculate loss (policy gradient)
            # print(states_tensor.shape)
            predicted_actions = meta_policy_model(states_tensor)
            # print(predicted_action.shape)
            loss = -torch.mean(predicted_actions * rewards_tensor.unsqueeze(-1))  # Maximize reward
            
            print(f"Episode {episode} | Loss: {loss.item()}")
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        state = next_state if not done else None

Episode 0 | Loss: 26.640233993530273
Episode 0 | Loss: -13.428032875061035
Episode 0 | Loss: -15.962553977966309
Episode 0 | Loss: -49.41343688964844
Episode 0 | Loss: 7.187632083892822
Episode 0 | Loss: -16.998750686645508
Episode 0 | Loss: 6.081997394561768
Episode 0 | Loss: -13.421334266662598
Episode 0 | Loss: -6.045792102813721
Episode 0 | Loss: -34.61580276489258
Episode 0 | Loss: 18.400583267211914
Episode 0 | Loss: -19.50225067138672
Episode 0 | Loss: -22.65096664428711
Episode 0 | Loss: -33.0982780456543
Episode 0 | Loss: -4.217139720916748
Episode 0 | Loss: -55.03659439086914
Episode 0 | Loss: -12.570832252502441
Episode 0 | Loss: -10.268170356750488
Episode 0 | Loss: -22.558368682861328
Episode 0 | Loss: 7.672202110290527
Episode 0 | Loss: -25.35881805419922
Episode 0 | Loss: -24.95396614074707
Episode 0 | Loss: -19.427040100097656
Episode 0 | Loss: -12.879790306091309
Episode 0 | Loss: -39.17497253417969
Episode 0 | Loss: -1.209252953529358
Episode 0 | Loss: -11.78465843200

### Training (Second Iteration)

In [19]:
def DRL_prediction(model, environment, deterministic=True):
        """make a prediction and get results"""
        test_env, test_obs = environment.get_sb_env()
        # print(test_obs)
        account_memory = None  # This help avoid unnecessary list creation
        actions_memory = None  # optimize memory consumption
        # state_memory=[] #add memory pool to store states
        state_memory = None
        rewards_memory = []

        test_env.reset()
        max_steps = len(environment.df.index.unique()) - 1

        for i in range(len(environment.df.index.unique())):
            action, _states = model.predict(test_obs, deterministic=deterministic)
            # account_memory = test_env.env_method(method_name="save_asset_memory")
            # actions_memory = test_env.env_method(method_name="save_action_memory")
            next_obs, rewards, dones, info = test_env.step(action)
            rewards_memory.append(rewards)
            # state_memory.append(test_obs)
            test_obs = next_obs
            if (
                i == max_steps - 1
            ):  # more descriptive condition for early termination to clarify the logic
                account_memory = test_env.env_method(method_name="save_asset_memory")
                actions_memory = test_env.env_method(method_name="save_action_memory")
            # add current state to state memory
                state_memory=test_env.env_method(method_name="save_state_memory")

            if dones[0]:
                print("hit end!")
                break
        # state_columns = [f"feature_{i}" for i in range(len(state_memory[0][0]))]
        # state_memory = pd.DataFrame([i[0] for i in state_memory], columns=state_columns)
        return account_memory[0], actions_memory[0], state_memory[0], np.array(rewards_memory)

In [20]:
ddpg_account_value, ddpg_actions, ddpg_states, ddpg_rewards = DRL_prediction(
    trained_ddpg, CustomStockTradingEnv(df=trade, **env_kwargs)
)

ppo_account_value, ppo_actions, ppo_states, ppo_rewards = DRL_prediction(
    trained_ppo, CustomStockTradingEnv(df=trade, **env_kwargs)
)

a2c_account_value, a2c_actions, a2c_states, a2c_rewards = DRL_prediction(
    trained_a2c, CustomStockTradingEnv(df=trade, **env_kwargs)
)

hit end!
hit end!
hit end!


In [21]:
decision_points = pd.concat([ddpg_states, ppo_states, a2c_states], axis=0)
decision_points.shape

(561, 291)

#### Training Dataset

In [22]:
env = CustomStockTradingEnv(df=trade, **env_kwargs)

In [23]:
dataset = []
for i in range(len(decision_points)):
    row = decision_points.iloc[i]
    state = row.values
    # print(state)
    action_best = None
    reward_best = -np.inf
    actions = []
    for model in [trained_ddpg, trained_ppo, trained_a2c]:
        action, _ = model.predict(state)
        actions.append(action)
        next_state, reward, done, info, _ = env.step(action, state)
        if reward > reward_best:
            reward_best = reward
            action_best = action
    input_vector = np.concatenate([np.concatenate(actions), state])
    dataset.append((input_vector, action_best))

# print(len(dataset))        
        
    
    
    

day: 187, episode: 0
begin_total_asset: 1000000.00
end_total_asset: 766530.72
total_reward: -233469.28
total_cost: 16405.41
total_trades: 4026
Sharpe: 0.063
day: 187, episode: 0
begin_total_asset: 1000000.00
end_total_asset: 766530.72
total_reward: -233469.28
total_cost: 16405.41
total_trades: 4026
Sharpe: 0.063
day: 187, episode: 0
begin_total_asset: 1000000.00
end_total_asset: 748254.55
total_reward: -251745.45
total_cost: 16405.41
total_trades: 4026
Sharpe: 0.063
day: 187, episode: 0
begin_total_asset: 1000000.00
end_total_asset: 748254.55
total_reward: -251745.45
total_cost: 16405.41
total_trades: 4026
Sharpe: 0.063
day: 187, episode: 0
begin_total_asset: 1000000.00
end_total_asset: 748254.55
total_reward: -251745.45
total_cost: 16405.41
total_trades: 4026
Sharpe: 0.063
day: 187, episode: 0
begin_total_asset: 1000000.00
end_total_asset: 800278.51
total_reward: -199721.49
total_cost: 16405.41
total_trades: 4026
Sharpe: 0.063
day: 187, episode: 0
begin_total_asset: 1000000.00
end_tot

In [41]:
D = TensorDataset(torch.FloatTensor(np.array([i[0] for i in dataset])), torch.FloatTensor(np.array([i[1] for i in dataset])))
loader = DataLoader(D, batch_size=64, shuffle=False)

In [52]:
meta_policy_model = MetaPolicy(input_dim, 128, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(meta_policy_model.parameters(), lr=0.0001)

In [51]:
for episode in range(100):
    episode_loss = 0
    for input, label in loader:
        optimizer.zero_grad()
        
        input_tensor = torch.FloatTensor(input).to(device)
        label_tensor = torch.FloatTensor(label).to(device)
        
        predicted_action = meta_policy_model(input_tensor)
        # print("input",input_tensor.shape)
        # print("predicted",predicted_action.shape)
        # print("label",label_tensor.shape)
        # break
        loss = criterion(predicted_action, label_tensor)
        loss.backward()
        optimizer.step()
        episode_loss += loss.item()
    # break
    print(f"Episode {episode} | Loss: {episode_loss}")

Episode 0 | Loss: 87159.080078125
Episode 1 | Loss: 86463.5458984375
Episode 2 | Loss: 85915.3876953125
Episode 3 | Loss: 84910.4326171875
Episode 4 | Loss: 84133.5302734375
Episode 5 | Loss: 83132.064453125
Episode 6 | Loss: 82158.1318359375
Episode 7 | Loss: 80764.2421875
Episode 8 | Loss: 80367.4638671875
Episode 9 | Loss: 80292.6083984375
Episode 10 | Loss: 79287.78271484375
Episode 11 | Loss: 80757.888671875
Episode 12 | Loss: 78662.564453125
Episode 13 | Loss: 77824.521484375
Episode 14 | Loss: 78242.82373046875
Episode 15 | Loss: 77609.4736328125
Episode 16 | Loss: 78080.0908203125
Episode 17 | Loss: 76925.08984375
Episode 18 | Loss: 75491.1875
Episode 19 | Loss: 75106.4140625
Episode 20 | Loss: 74164.09912109375
Episode 21 | Loss: 72330.6611328125
Episode 22 | Loss: 72309.697265625
Episode 23 | Loss: 73449.80322265625
Episode 24 | Loss: 72258.58642578125
Episode 25 | Loss: 69511.00830078125
Episode 26 | Loss: 67281.5419921875
Episode 27 | Loss: 68449.4267578125
Episode 28 | Los

### Training (Third Iteration)